# Heat Equation Porting Exercise

The following exercise will walk through the porting of a 3D heat equation code from Fortran to GT4Py.  The cell below contains a simple 3D heat equation code written in Fortran.  For this part of the exercise, the developer will do the following.

* Initialize Serialbox in the Fortran code
* Serialize the initial heat data `curr_Heat` and relevant variables to the Python/GT4Py port.
* Serialize the output heat data to compare to the Python/GT4Py port

After inserting the Serialbox calls, the developer can run the cell to create a source file called `heat.F90`.

In [1]:
%%writefile heat.F90

program heat

    implicit none

    integer :: ii, jj, kk, N, t_steps, curr_step

    real :: scale

    real, dimension(:,:,:), allocatable :: curr_Heat, future_Heat

    write(*,*) "Enter the Number of discretization cells N: "
    read(*,*) N
    write(*,*) "Enter number of time steps : "
    read(*,*) t_steps

    write(*,*) "N = ", N
    write(*,*) "Time steps = ", t_steps

    ! Note : Currently, the default domain is a cube, so the number of 
    !        discretization cells in x, y, and z are the same
    allocate(curr_Heat(N,N,N), future_Heat(N,N,N))

    !***Insert Serialbox calls here for initialization***

    curr_Heat   = 0.0
    future_Heat = 0.0

    scale = 0.1

    curr_Heat(2:N-1, 2:N-1, 2:N-1) = 1.0

    !***Insert Serialbox calls to create a savepoint to save initial starting data***

    do curr_step = 1,t_steps
        do kk = 2,N-1
            do jj = 2,N-1
                do ii = 2,N-1
                    future_Heat(ii,jj,kk) = curr_Heat(ii,jj,kk)              &
                                        + scale * (curr_Heat(ii-1,jj,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii+1,jj,kk))  &
                                        + scale * (curr_Heat(ii,jj-1,kk)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj+1,kk))  &
                                        + scale * (curr_Heat(ii,jj,kk-1)   &
                                                    -2.0*curr_Heat(ii,jj,kk) &
                                                    +curr_Heat(ii,jj,kk+1))
                enddo
            enddo
        enddo

        curr_Heat = future_Heat

    enddo
    
    !***Insert Serialbox calls to create a savepoint to write finalized data***

end program

Writing heat.F90


After adding the Serialbox calls, the developer can run the cell below to perform the following:

* Call `pp_ser.py` to substitute the Serialbox "directives" with Serialbox library calls in the Fortran code and output the result into a new Fortran source file called `s_heat.F90`
* Compile `s_heat.F90` and link with the Serialbox libraries into a binary called `HEAT`
* Run the `HEAT` binary 

Note that the code implementation has two inputs (size of the domain in cells and number of time steps) that must be listed on individual lines in the cell below when running the `HEAT` binary.  The developer can try different entries to help with debugging if necessary.

In [3]:
%%bash

[ -f HEAT ] && rm HEAT
[ -f s_heat.F90 ] && rm s_heat.F90

python3 ${SERIALBOX_ROOT}/python/pp_ser/pp_ser.py -s -v --output=s_heat.F90 heat.F90

gfortran -O3 -cpp -DSERIALIZE -o HEAT s_heat.F90 \
   -I${SERIALBOX_ROOT}/include \
    ${SERIALBOX_ROOT}/lib/libSerialboxFortran.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxC.a \
    ${SERIALBOX_ROOT}/lib/libSerialboxCore.a \
 -lpthread -lstdc++ -lstdc++fs
./HEAT
40
10


Processing file heat.F90
 Enter the Number of discretization cells N: 
 Enter number of time steps : 
 N =           40
 Time steps =           10


After ensuring that you are able to run the code and successfully serialize the data, you can start to write your Python/GT4Py port of the heat equation code in the cell below.  To compare solutions between the Python and the Fortran, use the `np.allclose()` function.

Here are a couple of things to keep in mind.

* Python array indices start at 0
* While it's not explicitly stated, the heat equation example has an implied boundary condition where the "surface" temperatures are set to 0

In [8]:
import serialbox as ser
import numpy as np
import gt4py
import gt4py.gtscript as gtscript
import gt4py.storage as gt_storage




Solution is valid!
